In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [2]:
dataset = pd.read_csv('dataset_NN.csv')
dataset.head()

,att1,att2,att3,att4,att5,att6,class
0,1.0,0.0,0.0,133.150861,1.311693,1620.221779,1
1,1.0,0.0,0.0,126.724861,1.302745,1609.334822,1
2,1.0,0.0,0.0,131.173861,1.319031,1568.978435,1
3,1.0,0.0,0.0,129.478861,1.270878,1695.055281,1
4,1.0,0.0,0.0,127.262861,1.329637,1647.720235,1


In [3]:
x = dataset.iloc[:,:-1].values    #feature attributes
y = dataset.iloc[:,-1:].values    #target attribute

In [4]:
def my_train_test_split(X, y, train_size):
    '''
    my own data splitting function
    '''
    arr_rand = np.random.rand(X.shape[0])
    split = arr_rand < np.percentile(arr_rand, train_size*100)

    X_Train = X[split]
    y_Train = y[split]
    X_Test =  X[~split]
    y_Test = y[~split]

    return X_Train, X_Test, y_Train, y_Test

def standardise_values(a):
    '''
    standardisation function, make data mean  = 0 and sd = 1
    '''
    mean = np.mean(a)
    sd = np.std(a)
    for i in range(len(a)):
        a[i] = (a[i]-mean)/sd
    return a

In [5]:
for i in range(len(x.T)):
    x.T[i] = standardise_values(x.T[i])

print(x[0],y[0])
print(x.shape)

[ 0.76340069 -1.51835049 -1.01989185 -0.93058101 -1.12578212 -1.20717246] [1]
(2000, 6)


In [45]:
#One Hot Encode y from (1400,1) to (1400,10)
def one_hot_encode(y):
    one_hot_y = np.zeros((y.shape[0],10))
    for i in range(y.shape[0]):
        one_hot_y[i][y[i] - 1] = 1

#     print(y[1000], one_hot_y[1000])
#     print(one_hot_y.shape)
    
    return one_hot_y

## Activations

In [6]:
def ReLU(x):
    return (abs(x.astype(float))+x.astype(float))/2

def ReLU_derivative(x):
    y=x
    np.piecewise(y,[ReLU(y)==0,ReLU(y)==y],[0,1])
    return y

def tanh(x):
    return np.tanh(x.astype(float))

def tanh_derivative(x):
    return 1-np.square(tanh(x))

def sigmoid(x):
    return 1/(1+np.exp(-x.astype(float)))

def sigmoid_derivative(x):
    return x*(1-x)

In [23]:
class NeuralNetwork:
    def __init__(self, x, y, alpha):
        self.input    = x
        self.weights1 = np.random.rand(self.input.shape[1], 100)    #first hidden layer has 4 nodes
        self.weights2 = np.random.rand(100,10)                      #output layer has 1 node
        self.y        = y
        self.output   = np.zeros(y.shape)
        self.alpha    = alpha

    
    def feedforward(self):
        self.layer1 = tanh(np.dot(self.input, self.weights1))
        self.output = sigmoid(np.dot(self.layer1, self.weights2))
    
    
    def backprop(self):
        m=len(self.input)
        dw2 = - (1/m) * np.dot(self.layer1.T, (self.y - self.output) * sigmoid_derivative(self.output))
        dw1 = -(1/m) * np.dot(self.input.T, (np.dot(2 * (self.y -self.output) * sigmoid_derivative(self.output), self.weights2.T) * tanh_derivative(self.layer1)))

        self.weights1 -= (self.alpha * dw1)
        self.weights2 -= (self.alpha * dw2)

    
    def train(self):
        self.feedforward()
        self.backprop()


    def calc_loss(self):
        m=len(self.input)
        loss = (1/m)*np.sum(np.square(self.y-self.output))
        return loss

    def predict(self,X):
        self.layert_1=tanh(np.dot(X,self.weights1))
        return sigmoid(np.dot(self.layert_1,self.weights2))


In [48]:
def accuracy(y_pred, y_orig):
        total = len(y_pred)
        correct = 0
        for i in range(total):
#       print(y_pred[i], y_orig[i])
            if y_pred[i] == y_orig[i]:
                correct = correct + 1
        return (correct / total)  * 100

In [47]:
X_train, X_test, y_train, y_test = my_train_test_split(x, y, 0.7)
one_hot_y_train = one_hot_encode(y_train)
one_hot_y_test = one_hot_encode(y_test)

NN = NeuralNetwork(X_train, one_hot_y_train, 0.1)
print(NN.weights1.shape)
print(NN.weights2.shape)
print(NN.output.shape)
iterations = 10000

for i in range(iterations):
    NN.train()
    if i % 1000 == 0: 
        print ("for iteration # " + str(i) + "\n")
#         print(NN.weights1,"\n")
#         print(NN.weights2,"\n")
        print ("Loss: \n" + str(NN.calc_loss())) # mean sum squared loss
        print ("\n")

print(NN.output.shape)

y_predict_train = NN.output
output_train = (np.argmax(y_predict_train, axis = 1) + 1).reshape(output_train.shape[0],1)
y_predict_test = NN.predict(X_test)
output_test = (np.argmax(y_predict_test, axis = 1) + 1).reshape(output_test.shape[0],1)

print("Output shape is = ",output_train.shape)
print("y_train shape is = ",y_train.shape)

train_accuracy = accuracy(output_train, y_train)
test_accuracy = accuracy(output_test, y_test)
cost = NN.calc_loss()

#----------------Displaying Final Metrics--------------------------#
print("MSE_Cost: {} , Train_Accuracy: {} , Test_Accuracy: {} ".format(cost,train_accuracy,test_accuracy))

(6, 100)
(100, 10)
(1400, 10)
for iteration # 0

Loss: 
4.29736404172862


for iteration # 1000

Loss: 
4.07347307484459


for iteration # 2000

Loss: 
2.910084012851308


for iteration # 3000

Loss: 
2.6136622880659965


for iteration # 4000

Loss: 
0.8396442066910724


for iteration # 5000

Loss: 
0.5698467094667942


for iteration # 6000

Loss: 
0.5072902911409137


for iteration # 7000

Loss: 
0.49140571361470553


for iteration # 8000

Loss: 
0.9026874824331915


for iteration # 9000

Loss: 
0.6370573586216374


(1400, 10)
Output shape is =  (1400, 1)
y_train shape is =  (1400, 1)
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]
[1] [1]

[8] [8]
[8] [8]
[8] [8]
[8] [8]
[5] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[5] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[4] [8]
[8] [8]
[5] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[5] [8]
[3] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[3] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[5] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[2] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[5] [8]
[8] [8]
[5] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[5] [8]
[8] [8]
[5] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[5] [8]
[8] [8]
[3] [8]
[8] [8]
[3] [8]
[8] [8]
[8] [8]
[8] [8]
[3] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[5] [8]
[8] [8]
[8] [8]
[8] [8]
[3] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[8] [8]
[4] [8]
[8] [8]
[3] [8]
[8] [8]
